**Set-up**


In [12]:
%%capture
!pip install -r requirements


In [13]:
# Import Packages
## for data and preprocessing
import pandas as pd
from sklearn.preprocessing import StandardScaler

## for model fitting
import lightgbm as lgb
import sklearn.metrics as metric

## for hyperparameter optimization
import optuna

In [14]:
train = pd.read_csv('./data/california_housing_train.csv')
test = pd.read_csv('./data/california_housing_test.csv')

names = train.columns

scaler = StandardScaler()
train = pd.DataFrame(scaler.fit_transform(train),columns=names)
test = pd.DataFrame(scaler.transform(test), columns=names)


X_train = train.drop(['median_house_value'],axis=1)
X_test  = test.drop(['median_house_value'],axis=1)
y_train = train.median_house_value
y_test  = test.median_house_value

# **OPTUNA**

## **General Overview**

Optuna optimizes any objective function. This objective function takes a set of arguments (e.g., hyperparameters) and returns a single value (e.g., validation score).  

In Optuna, we create a **study**. A study is defined by the objective function and the hyperparameter space and, thus, defines the scope and purpose of our optimization exercise.   
Each study consists of a set of **Trials**. Each trial is, thus, a single selection from the hyperparameter space for which we evaluate the objective function. Every next trial builds on the previous one (i.e., an iterative optimization process).

The optimization algorithm helps in picking the next trial to evaluate in a smart(er) way, until we find the optimal value.

In practice, every hyperparameter optimization exercise consist of 4 steps:

* define a function which **trains a model** and **returns the validation score**

* define the **hyperparameter space** through which the optimization algorithm can search (trials are instances/realizations of this space)

* create a **study**, which describes the optimization exercise: 
    * *Direction* : 
        * minimize: for (Root) Mean Squared Errors, minus-log-likelihood, ... (the lower, the better)
        * maximize: r2_score, auc, accuracy, precision, recall, f1_score, ... (the higher, the better)
    * *Sampler* : the chosen optimization technique **(Optimization)**
    * *Pruner* : early stopping of unpromising trials **(Steroids)**

* **optimize** the study using different trials in a smart way **(worker function)**


Firstly, we need to realize that our time is also limited. In order to limit our waiting time (and computing time), we set a maximum number of trials to evaluate (i.e., maximum number of iterations). 

In [15]:
N_TRIALS = 200

### Step 1
We define a function which takes a hyperparameter configuration (params;  which is defined later) as the argument.  
Then this function takes our data and trains a machine learning model. In this example, we train a lightgbm model, which can take a lot of interesting hyperparameters to illustrate tuning. Any model architecture can work here (e.g., xgboost, random forest, neural networks, ...).  
Lastly, we make some predictions on our test (or validation) set and compute the validation score. In this example, we use the Root Mean Squared Error (RMSE), but again any validation metric is viable.

In [16]:
def train_evaluate(params):
    '''Train a model using your dataset and return the validation score.'''
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    # Train a Model
    model = lgb.train(params, train_data,
                      num_boost_round=params['NUM_BOOST_ROUND'],
                      early_stopping_rounds=params['EARLY_STOPPING_ROUNDS'],
                      valid_sets=[test_data],
                      valid_names=['valid'],
                      )
    # Evaluate the model
    preds = model.predict(test_data,num_iteration=model.best_iteration)
    truth = test_data.get_label()
    score = metric.mean_squared_error(truth, preds, squared=False)
      
    #score = model.best_score['valid']['rmse']
    # Return the validation score
    return score

### Step 2
We define the objective function of our optimization exercise, which takes a trial as argument.  
In this function, we first define the parameter space. This parameter space is a dictionary defining each hyperparameter of interest. For each hyperparameter, we use the $trial.suggest$ functionality to define the domain from which we can sample values for the hyperparameters. In a Bayesian way, think about this as our prior (hyper)parameter distribution. We can use various distributions:
- trial.suggest_loguniform for floating point hyperparameters between two bounds favoring smaller values,
- trial.suggest_float for floating point hyperparameters between two bounds
- trial.suggest_int for integer hyperparameters between two bounds and a step-size,
- trial.suggest_uniform for uniformly distributed hyperparameters between two bounds,
- trial.suggest_discrete_uniform for uniformly distrubuted hyperparameters between two bounds but with additional step-size,
- ...

After defining the hyperparameter space, we apply the previously defined function to train a model and return the validation score.  
At this stage, we will also check whether the score should be pruned or not (depending on whether a pruning strategy was specified).

In [17]:
def objective(trial):
    '''
    Define the Hyperparameter Space from which to sample a configuration.
    Then train a model and output the validation score (see Step 1).
    '''
    # Define the Hyper-parameter Space
    params = {'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
              'max_depth': trial.suggest_int('max_depth', 1, 30, 1),
              'num_leaves': trial.suggest_int('num_leaves', 2, 100),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
              'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
              'subsample': trial.suggest_discrete_uniform('subsample', 0.1, 1.0,.1),
              'colsample_by_tree': 1,
              'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
              'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
              'NUM_BOOST_ROUND': 200,
              'EARLY_STOPPING_ROUNDS': 20,
              'objective': 'rmse',
              }
              
    # Train the model and return the validation score
    score = train_evaluate(params)
    
    #Check Pruning
    trial.report(score,1)
    if trial.should_prune():
        raise optuna.TrialPruned()

    # Return the validation score
    return score

### Step 3
We create the study object, in which we describe the optimization exercise by means of: 
- the direction
- the sampler
- the pruning

* the *Direction* of our optimization: 
        * minimize: for (Root) Mean Squared Errors, minus-log-likelihood, ... (the lower, the better)
        * maximize: r2_score, auc, accuracy, precision, recall, f1_score, ... (the higher, the better)
* the *Sampler* which is our optimization technique: 
        * GridSampler, applies a Grid Search on a predefined grid (extra arguments required!)
        * RandomSampler, applies Random Search on the parameter space
        * CmaEsSampler, applies a Covariance Matrix Adaptation Evolutionary Search algorithm
        * TPESampler, is the default option, which applies a Tree-structured Parzen Estimator algorithm

* the *Pruner* which is our pruning strategy to quickly stop unpromising trials:
        * NopPruner, does not prune any trials
        * MedianPruner, prunes trials that are worst than the median of previous trials
        * SuccessiveHalvingPruner, uses Asynchronous Successive Halving (prune half of the least performing trials)
        * HyperbandPruner, uses the Hyperband pruning strategy

In [18]:
study = optuna.create_study(
    direction = 'minimize',                         
    sampler = optuna.samplers.RandomSampler(),      
    pruner = optuna.pruners.NopPruner()            
    )

[I 2021-11-05 18:06:38,779] A new study created in memory with name: no-name-96500fba-67ea-4e69-a947-98aeedd080fd


### Step 4
We call the optimize function on our study object to start the optimization process. 

In [19]:
%%script false --no-raise-error
study.optimize(objective, n_trials=N_trials)

Couldn't find program: 'false'


## Example of Several Optimization Strategies

Uncomment the following line if you want to suppress all output of the optuna sampler.

In [20]:
#optuna.logging.set_verbosity(optuna.logging.WARNING)

Next, we define out $train\_evaluate$ and our $objective$ functions. 

In [21]:
def train_evaluate(params):
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    # Train a Model
    model = lgb.train(params, train_data,
                      num_boost_round=params['NUM_BOOST_ROUND'],
                      early_stopping_rounds=params['EARLY_STOPPING_ROUNDS'],
                      valid_sets=[test_data],
                      valid_names=['valid'],
                      )
    # Evaluate the model 
    preds = model.predict(X_test,num_iteration=model.best_iteration)
    truth = test_data.get_label()
    score = metric.mean_squared_error(truth, preds, squared=False)
    # Return the validation score
    return score

def objective(trial):
    # Define the Hyper-parameter Space
    params = {'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
              'max_depth': trial.suggest_int('max_depth', 1, 50),
              'num_leaves': trial.suggest_int('num_leaves', 2, 200),
              'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
              'subsample': trial.suggest_discrete_uniform('subsample', 0.1, 1.0, .1),
              'colsample_by_tree': 1,
              'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
              'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
              'bagging_fraction':trial.suggest_uniform('bagging_fraction', 0, 1),
              'bagging_freq':trial.suggest_int('bagging_freq',0,10),
              'NUM_BOOST_ROUND': 200,
              'EARLY_STOPPING_ROUNDS': 20,
              'objective': 'rmse',
              }
    # Train the model and return the validation score
    score = train_evaluate(params)
    
    #Check Pruning
    trial.report(score,200)
    if trial.should_prune():
      raise optuna.TrialPruned()
    
    # Return the validation score
    return score

### **Grid Search**

A grid search does not really look at the hyperparameter space, but rather takes a search space with discrete lists of hyperparameter values into account.
In this example, we search over a small grid of 4*4*4 hyperparameters (total size of the grid: 48 possibilities).

In [23]:
%%time
%%capture

def objective_grid(trial):
    # Define the Hyper-parameter Space
    params = {'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
              'max_depth': trial.suggest_int('max_depth', 1, 50),
              'num_leaves': trial.suggest_int('num_leaves', 2, 200),
              'NUM_BOOST_ROUND': 200,
              'EARLY_STOPPING_ROUNDS': 20,
              'objective': 'rmse',
              'verbose': -1,
              }
    
    score = train_evaluate(params)
    return score

search_space = {'learning_rate': [0.01, 0.10, 0.50],
              'max_depth': [1, 10, 20, 30],
              'num_leaves': [2, 10, 20, 100]}

study_gridsearch = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.GridSampler(search_space),
    pruner = optuna.pruners.NopPruner() 
    )

study_gridsearch.optimize(objective_grid, n_trials=N_TRIALS)

[I 2021-11-05 18:08:37,706] A new study created in memory with name: no-name-4554cb7e-f965-4581-8fc8-11f728f725db
[I 2021-11-05 18:08:37,810] Trial 0 finished with value: 0.7470006743845352 and parameters: {'learning_rate': 0.01, 'max_depth': 1, 'num_leaves': 100}. Best is trial 0 with value: 0.7470006743845352.
[I 2021-11-05 18:08:38,348] Trial 1 finished with value: 0.47375380713319415 and parameters: {'learning_rate': 0.01, 'max_depth': 20, 'num_leaves': 100}. Best is trial 1 with value: 0.47375380713319415.
[I 2021-11-05 18:08:38,417] Trial 2 finished with value: 0.5261623792105229 and parameters: {'learning_rate': 0.5, 'max_depth': 1, 'num_leaves': 100}. Best is trial 1 with value: 0.47375380713319415.
[I 2021-11-05 18:08:38,603] Trial 3 finished with value: 0.43097071502431894 and parameters: {'learning_rate': 0.5, 'max_depth': 10, 'num_leaves': 100}. Best is trial 3 with value: 0.43097071502431894.
[I 2021-11-05 18:08:38,673] Trial 4 finished with value: 0.5988083376612268 and p

Wall time: 7.51 s


In [24]:
gridsearch = {'score': study_gridsearch.best_value, 'params': study_gridsearch.best_params}
print(gridsearch)

{'score': 0.4009600440564793, 'params': {'learning_rate': 0.1, 'max_depth': 10, 'num_leaves': 100}}


## **Random Search**

In [25]:
%%time
%%capture

study_randomsearch = optuna.create_study(
    direction = 'minimize',
    sampler = optuna.samplers.RandomSampler(),
    pruner = optuna.pruners.NopPruner() 
    )

study_randomsearch.optimize(objective, n_trials=N_TRIALS)

[I 2021-11-05 18:08:50,882] A new study created in memory with name: no-name-91bcaa74-c5c7-4d13-b707-fe78db373059
[I 2021-11-05 18:08:51,397] Trial 0 finished with value: 0.42836543707044783 and parameters: {'learning_rate': 0.08243238042068547, 'max_depth': 17, 'num_leaves': 94, 'feature_fraction': 0.4715368132122373, 'subsample': 0.30000000000000004, 'lambda_l1': 6.379255534236382, 'lambda_l2': 8.397918493009744, 'bagging_fraction': 0.42680580086407505, 'bagging_freq': 9}. Best is trial 0 with value: 0.42836543707044783.
[I 2021-11-05 18:08:51,563] Trial 1 finished with value: 0.4620424605810551 and parameters: {'learning_rate': 0.3312210706075792, 'max_depth': 48, 'num_leaves': 163, 'feature_fraction': 0.9494189702146603, 'subsample': 0.7000000000000001, 'lambda_l1': 5.255637811692717, 'lambda_l2': 3.0081457061855787, 'bagging_fraction': 0.09612805196782426, 'bagging_freq': 3}. Best is trial 0 with value: 0.42836543707044783.
[I 2021-11-05 18:08:52,132] Trial 2 finished with value: 

Wall time: 1min 22s


In [26]:
randomsearch = {'score': study_randomsearch.best_value, 'params': study_randomsearch.best_params}
print(randomsearch)

{'score': 0.39231776491712855, 'params': {'learning_rate': 0.06488210316955564, 'max_depth': 32, 'num_leaves': 198, 'feature_fraction': 0.6423956698204506, 'subsample': 0.4, 'lambda_l1': 0.8859296979133158, 'lambda_l2': 3.463731445929307, 'bagging_fraction': 0.9901469862241185, 'bagging_freq': 3}}


## **CMAES**

In [27]:
%%time
%%capture

study_cmaes = optuna.create_study(
    direction = 'minimize',
    sampler = optuna.samplers.CmaEsSampler(),
    pruner = optuna.pruners.MedianPruner()  
    )

study_cmaes.optimize(objective, n_trials=N_TRIALS)

[I 2021-11-05 18:10:46,569] A new study created in memory with name: no-name-6c24707d-d96e-4969-9a1c-e3b94182078a
[I 2021-11-05 18:10:47,082] Trial 0 finished with value: 0.41082541783973225 and parameters: {'learning_rate': 0.06477642923586423, 'max_depth': 23, 'num_leaves': 86, 'feature_fraction': 0.6651807892596312, 'subsample': 0.4, 'lambda_l1': 3.308617712328777, 'lambda_l2': 6.83331405936109, 'bagging_fraction': 0.37339002283766853, 'bagging_freq': 7}. Best is trial 0 with value: 0.41082541783973225.
[I 2021-11-05 18:10:47,623] Trial 1 finished with value: 0.42580999910883927 and parameters: {'learning_rate': 0.09130014664860443, 'max_depth': 26, 'num_leaves': 101, 'feature_fraction': 0.3285094497412938, 'subsample': 0.5, 'lambda_l1': 5.1719662229914345, 'lambda_l2': 4.943066283594739, 'bagging_fraction': 0.6780443456998841, 'bagging_freq': 5}. Best is trial 0 with value: 0.41082541783973225.
[I 2021-11-05 18:10:48,116] Trial 2 finished with value: 0.42449686498571304 and paramet

Wall time: 2min 48s


In [28]:
cmaessearch = {'score': study_cmaes.best_value, 'params': study_cmaes.best_params}
print(cmaessearch)

{'score': 0.3956091733845269, 'params': {'learning_rate': 0.08258532700712243, 'max_depth': 25, 'num_leaves': 101, 'feature_fraction': 0.6409066363317579, 'subsample': 0.8, 'lambda_l1': 4.676965102421771, 'lambda_l2': 5.046539801167314, 'bagging_fraction': 0.9861621442714299, 'bagging_freq': 5}}


## **BOHB**

In [29]:
%%time
%%capture

study_bohb = optuna.create_study(
    direction = 'minimize',
    sampler = optuna.samplers.TPESampler(),
    pruner = optuna.pruners.HyperbandPruner()
    )

study_bohb.optimize(objective, n_trials=N_TRIALS)

[I 2021-11-05 18:14:11,489] A new study created in memory with name: no-name-d24deefc-fe88-4c28-9863-51c4b9be4885
[I 2021-11-05 18:14:11,767] Trial 0 finished with value: 0.7447051620827624 and parameters: {'learning_rate': 0.013458154326238021, 'max_depth': 34, 'num_leaves': 130, 'feature_fraction': 0.1377126691338408, 'subsample': 0.6, 'lambda_l1': 7.945249477967162, 'lambda_l2': 7.118335477446801, 'bagging_fraction': 0.6222336184667068, 'bagging_freq': 2}. Best is trial 0 with value: 0.7447051620827624.
[I 2021-11-05 18:14:12,185] Trial 1 finished with value: 0.7477699870782398 and parameters: {'learning_rate': 0.012631784215464487, 'max_depth': 12, 'num_leaves': 112, 'feature_fraction': 0.14263943782996275, 'subsample': 0.7000000000000001, 'lambda_l1': 0.8876704761045584, 'lambda_l2': 2.1056875730154925, 'bagging_fraction': 0.27402658697064053, 'bagging_freq': 4}. Best is trial 0 with value: 0.7447051620827624.
[I 2021-11-05 18:14:12,465] Trial 2 finished with value: 0.444587125919

Wall time: 2min 23s


In [30]:
bohbsearch = {'score': study_bohb.best_value, 'params': study_bohb.best_params}
print(bohbsearch)

{'score': 0.39244275830550013, 'params': {'learning_rate': 0.03266296527591634, 'max_depth': 17, 'num_leaves': 196, 'feature_fraction': 0.6702961486900811, 'subsample': 0.2, 'lambda_l1': 0.050210033547134714, 'lambda_l2': 3.62466188456434, 'bagging_fraction': 0.7575518974762833, 'bagging_freq': 6}}


The final overview of our results shows that BOHB performs best. 
In this example, the differences are not extreme, which is mostly due to the very stylized example. In many other cases, the gains of hyperparameter optimization are considerable.

In [31]:
pd.DataFrame([gridsearch['score'],randomsearch['score'],cmaessearch['score'],bohbsearch['score']],index=['Grid','Random','CMAES','BOHB'],columns=['RMSE'])

,RMSE
Grid,0.400960
Random,0.392318
CMAES,0.395609
BOHB,0.392443


### **Visualization**

In [ ]:
#History: 
trials_df = study_bohb.trials_dataframe()
trials_df

In [ ]:
optuna.visualization.plot_optimization_history(study_bohb)

In [ ]:
optuna.visualization.plot_param_importances(study_bohb)

In [ ]:
optuna.visualization.plot_slice(study_bohb)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study_bohb)